In [1]:
import pandas as pd
import plotly.graph_objects as go
import utils

In [2]:
plot_cols = ["ENTRY", "STOPLOSS", "TAKEPROFIT"]
plot_colors = ["#043ef9", "#eb5334", "#34eb37"]

def plot_candles(df_plot):
    fig = go.Figure()
    fig.add_trace(go.Candlestick(
        x = df_plot.time, open = df_plot.mid_o, high = df_plot.mid_h, low = df_plot.mid_l, close = df_plot.mid_c,
        line=dict(width=1), opacity=1,
        increasing_fillcolor="#23A06B",
        decreasing_fillcolor="#CC2E3C",
        increasing_line_color="#2EC886",
        decreasing_line_color="#FF3A4C"
    ))
    for i in range(0,3):
        fig.add_trace(go.Scatter(
            x=df_buys.time,
            y=df_buys[plot_cols[i]],
            mode='markers',
            marker=dict(color=plot_colors[i],size=12)
        ))
    for i in range(0,3):    
        fig.add_trace(go.Scatter(
            x=df_sells.time,
            y=df_sells[plot_cols[i]],
            mode='markers',
            marker=dict(color=plot_colors[i],size=12)
        ))
    fig.update_layout(width=1000,height=400, font = dict(size=10, color = "#e1e1e1"), paper_bgcolor="#1e1e1e", plot_bgcolor="#1e1e1e")
    fig.update_xaxes(gridcolor = "#1f292f", showgrid=True, fixedrange=True, rangeslider=dict(visible=False), rangebreaks=[dict(bounds=["sat","mon"])])
    fig.update_yaxes(gridcolor= "#1f292f", showgrid=True)


    fig.show()

In [3]:
pair = "USD_JPY"
granularity = 'H4'
df_raw = pd.read_pickle(utils.get_his_data_filename(pair, granularity))

In [4]:
non_cols = ['time', 'volume']
mod_cols = [x for x in df_raw.columns if x not in non_cols]
df_raw[mod_cols] = df_raw[mod_cols].apply(pd.to_numeric)

In [5]:
df_raw.head()

,time,volume,mid_o,mid_h,mid_l,mid_c,bid_o,bid_h,bid_l,bid_c,ask_o,ask_h,ask_l,ask_c
0,2019-01-01T22:00:00.000000000Z,2616,109.590,109.726,109.533,109.598,109.540,109.718,109.493,109.591,109.640,109.736,109.557,109.606
1,2019-01-02T02:00:00.000000000Z,2710,109.600,109.613,109.299,109.304,109.593,109.606,109.291,109.297,109.608,109.620,109.307,109.312
2,2019-01-02T06:00:00.000000000Z,6642,109.302,109.348,108.710,108.943,109.294,109.341,108.702,108.937,109.310,109.354,108.717,108.949
3,2019-01-02T10:00:00.000000000Z,6613,108.946,109.284,108.822,109.191,108.939,109.278,108.813,109.185,108.952,109.291,108.829,109.197
4,2019-01-02T14:00:00.000000000Z,7780,109.189,109.464,109.177,109.442,109.182,109.457,109.170,109.435,109.196,109.471,109.184,109.449


In [6]:
df_raw.columns

Index(['time', 'volume', 'mid_o', 'mid_h', 'mid_l', 'mid_c', 'bid_o', 'bid_h',
       'bid_l', 'bid_c', 'ask_o', 'ask_h', 'ask_l', 'ask_c'],
      dtype='object')

In [7]:
SLOSS = 0.4
TPROFIT = 0.8
ENTRY_PRC = 0.1

def direction(row):
    if row.mid_c > row.mid_o:
        return 1
    return -1
def get_signal(row):
    if row.mid_h_prev > row.mid_h and row.mid_l_prev < row.mid_l:
        return row.DIRECTION_prev
    return 0

def get_entry_stop(row):
    if row.SIGNAL == 1:
        return (row.RANGE_prev * ENTRY_PRC) + row.ask_h_prev
    elif row.SIGNAL == -1:
        return row.bid_l_prev - (row.RANGE_prev * ENTRY_PRC)
    else:
        return 0

def get_stop_loss(row):
    if row.SIGNAL == 1:
        return row.ENTRY - (row.RANGE_prev * SLOSS)
    elif row.SIGNAL == -1:
        return row.ENTRY + (row.RANGE_prev * SLOSS)
    else:
        return 0


def get_take_profit(row):
    if row.SIGNAL == 1:
        return row.ENTRY + (row.RANGE_prev * TPROFIT)
    elif row.SIGNAL == -1:
        return row.ENTRY - (row.RANGE_prev * TPROFIT)
    else:
        return 0

In [8]:
df = df_raw.copy()
df['RANGE'] = df.mid_h - df.mid_l
df['mid_h_prev'] = df.mid_h.shift(1)
df['mid_l_prev'] = df.mid_l.shift(1)
df["ask_h_prev"] = df.ask_h.shift(1)
df['bid_l_prev'] = df.bid_l.shift(1)
df['RANGE_prev'] = df.RANGE.shift(1)
df['DIRECTION'] = df.apply(direction, axis = 1)
df['DIRECTION_prev'] = df.DIRECTION.shift(1).fillna(0).astype(int)
df.dropna(inplace=True)
df['SIGNAL'] = df.apply(get_signal, axis=1)
df.reset_index(drop=True, inplace=True)
df['ENTRY'] = df.apply(get_entry_stop, axis = 1)
df['STOPLOSS'] = df.apply(get_stop_loss, axis = 1)
df['TAKEPROFIT'] = df.apply(get_take_profit, axis = 1)

In [9]:
df.head()

,time,volume,mid_o,mid_h,mid_l,mid_c,bid_o,bid_h,bid_l,bid_c,...,mid_l_prev,ask_h_prev,bid_l_prev,RANGE_prev,DIRECTION,DIRECTION_prev,SIGNAL,ENTRY,STOPLOSS,TAKEPROFIT
0,2019-01-02T02:00:00.000000000Z,2710,109.600,109.613,109.299,109.304,109.593,109.606,109.291,109.297,...,109.533,109.736,109.493,0.193,-1,1,0,0.0000,0.0000,0.0000
1,2019-01-02T06:00:00.000000000Z,6642,109.302,109.348,108.710,108.943,109.294,109.341,108.702,108.937,...,109.299,109.620,109.291,0.314,-1,-1,0,0.0000,0.0000,0.0000
2,2019-01-02T10:00:00.000000000Z,6613,108.946,109.284,108.822,109.191,108.939,109.278,108.813,109.185,...,108.710,109.354,108.702,0.638,1,-1,-1,108.6382,108.8934,108.1278
3,2019-01-02T14:00:00.000000000Z,7780,109.189,109.464,109.177,109.442,109.182,109.457,109.170,109.435,...,108.822,109.291,108.813,0.462,1,1,0,0.0000,0.0000,0.0000
4,2019-01-02T18:00:00.000000000Z,4395,109.444,109.456,108.869,108.887,109.437,109.450,108.837,108.837,...,109.177,109.471,109.170,0.287,-1,1,0,0.0000,0.0000,0.0000


In [10]:
class Trade():
    def __init__(self, row):
        self.candle_date = row.time
        self.direction = row.SIGNAL
        self.entry = row.ENTRY
        self.TP = row.TAKEPROFIT
        self.SL = row.STOPLOSS
        self.running = False
        self.result = None
        self.stopped = None

    def update(self, row):
        if self.running == True:
            self.update_result(row)
        else:
            self.check_entry(row)    
    
    def check_entry(self, row):
        if self.direction == 1 and row.mid_c >= self.entry or self.direction == -1 and row.mid_c <= self.entry:
            self.index = row.name
            self.opened = row.time
            self.running = True

    def update_result(self, row):
        if self.direction == 1:
            if row.mid_c >= self.TP:
                self.result = 2.0
            elif row.mid_c <= self.SL:
                self.result = -1.0
        else:
            if row.mid_c <= self.TP:
                self.result = 2.0            
            elif row.mid_c >= self.SL:
                self.result = -1.0   
        
        if self.result is not None:
            self.running = False
            self.stopped = row.time

In [11]:
open_trades = []
closed_trades = []

for index, row in df.iterrows():
    for ot in open_trades:
        ot.update(row)
        if ot.stopped is not None:
            closed_trades.append(ot)
    
    open_trades = [x for x in open_trades if x.stopped is None]

    if row.SIGNAL != 0:
        open_trades = [x for x in open_trades if x.running == True]
        open_trades.append(Trade(row))

In [12]:
df_trades = pd.DataFrame.from_dict([vars(x) for x in closed_trades])

In [13]:
df[df.SIGNAL != 0].to_pickle("USD_JPY_H4_trades.pkl")

In [14]:
df[df.SIGNAL != 0].shape
df_plot = df.iloc[200:250]
df_buys = df_plot[df_plot.SIGNAL == 1]
df_sells = df_plot[df_plot.SIGNAL == -1]


In [15]:
plot_candles(df_plot)

In [16]:
df_trades["result"].sum()

192.0